In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")


In [ ]:

def split(title, df):
    return df.drop(title, axis=1), df[title]

In [ ]:
x, y = split("Exited", train_df.drop(["id", "CustomerId"], axis=1))

In [61]:

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

prediction_list = []
accuracies = []
for train_idx, test_idx in kf.split(x, y):
    x_train, x_val = x.iloc[train_idx], x.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    model = CatBoostClassifier(cat_features=[0, 2, 3])
    model.fit(x_train, y_train, cat_features=[0, 2, 3], verbose=100) 

    # For testing
    predictions = model.predict(x_val)
    accuracy = accuracy_score(y_val, predictions)
    accuracies.append(accuracy)

    # For submitting
    final_predictions = model.predict_proba(test_df.drop(["id", "CustomerId"], axis=1))[:, 1]
    prediction_list.append(final_predictions)

print(accuracies)
print(np.mean(accuracies))

Learning rate set to 0.082881
0:	learn: 0.6159828	total: 102ms	remaining: 1m 41s
100:	learn: 0.3120348	total: 9.99s	remaining: 1m 28s
200:	learn: 0.3074583	total: 20s	remaining: 1m 19s
300:	learn: 0.3040556	total: 30.2s	remaining: 1m 10s
400:	learn: 0.3012404	total: 39.7s	remaining: 59.3s
500:	learn: 0.2986759	total: 49.2s	remaining: 49s
600:	learn: 0.2964553	total: 58.7s	remaining: 39s
700:	learn: 0.2942208	total: 1m 8s	remaining: 29.1s
800:	learn: 0.2921447	total: 1m 17s	remaining: 19.3s
900:	learn: 0.2901569	total: 1m 27s	remaining: 9.59s
999:	learn: 0.2881942	total: 1m 39s	remaining: 0us
Learning rate set to 0.082881
0:	learn: 0.6158024	total: 102ms	remaining: 1m 42s
100:	learn: 0.3123253	total: 9.87s	remaining: 1m 27s
200:	learn: 0.3075360	total: 19.8s	remaining: 1m 18s
300:	learn: 0.3040213	total: 29.8s	remaining: 1m 9s
400:	learn: 0.3011413	total: 40.4s	remaining: 1m
500:	learn: 0.2987135	total: 50.5s	remaining: 50.3s
600:	learn: 0.2964684	total: 1m	remaining: 40.1s
700:	learn: 

In [ ]:
sample_df = pd.read_csv("../sample_submission.csv")
sample_df['Exited'] = np.mean(final_predictions)

In [ ]:
sample_df.to_csv("lateSubmission.csv", index=False)